In [14]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [15]:
test_err = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\preprocessed data\new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\preprocessed data\new_test_quality.csv', parse_dates=['time'])
train = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\train_case1.csv')

In [16]:
train

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.442781,0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,0.442781,0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,0.272376,0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,0.442781,0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,0.442781,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.316156,-0.340504,-0.239957,-0.053688,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.491089,0.019092,0.482907,-2.965326,0
14996,-0.387084,-0.340504,-0.239957,-0.414143,-0.062749,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-4.498974,0
14997,-0.078337,0.704060,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,0.034927,-0.074159,0.628662,-0.114398,-0.080131,0.115262,0.019092,0.764726,0.442781,0
14998,-0.333983,-0.156620,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,...,-0.045467,-0.074159,0.628662,-0.114398,-0.080131,-0.528213,-0.336841,-0.644371,-1.090867,0


In [17]:
df1 = test_err.copy()
df1

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1


In [18]:
# counts column 추가
df1['counts'] = 1
df1

,user_id,time,model_nm,fwver,errtype,errcode,counts
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2,1
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1,1
...,...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0,1
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1,1


In [19]:
# multiple_model
model_df =  test_err[['user_id', 'model_nm']]
model_df = model_df.drop_duplicates(subset=None, keep='first')

model_num = model_df.user_id.value_counts(sort=False)
user_has_multiple_models = model_num > 1

user_has_multiple_models = user_has_multiple_models.astype(int)
user_has_multiple_models = user_has_multiple_models.to_frame(name='multiple_model')
user_has_multiple_models.head(7)

,multiple_model
30000,1
30001,0
30002,0
30003,0
30004,1
30005,0
30006,0


In [20]:
# multiple_fwver
fwver_df =  test_err[['user_id', 'fwver']]
fwver_df = fwver_df.drop_duplicates(subset=None, keep='first')

fwver_num = fwver_df.user_id.value_counts(sort=False)
user_has_multiple_fwvers = fwver_num > 1

user_has_multiple_fwvers = user_has_multiple_fwvers.astype(int)
user_has_multiple_fwvers = user_has_multiple_fwvers.to_frame(name='multiple_fwver')
user_has_multiple_fwvers.head(7)

,multiple_fwver
30000,1
30001,0
30002,1
30003,1
30004,1
30005,0
30006,0


In [21]:
# 1. one-hot-encoding
df1.drop(columns=['time', 'errcode'], inplace=True)
df1 = pd.get_dummies(df1, columns=['model_nm', 'fwver', 'errtype'])
df1

,user_id,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30000,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15527216,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15527217,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15527218,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15527219,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# period
test_err['day'] = test_err.time.dt.day
test_err['hour'] = test_err.time.dt.hour
test_err.head()

,user_id,time,model_nm,fwver,errtype,errcode,day,hour
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1,1,3
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2,1,3
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1,1,3
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1,1,3
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1,1,3


In [23]:
user_day_df = test_err.groupby('user_id')['day'].unique().to_frame().reset_index()
user_day_df['period'] = 0 

for i in range(len(user_day_df.index)):
   user_day_df['period'][i] = len(user_day_df['day'][i])

user_day_df

,user_id,day,period
0,30000,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",29
1,30001,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",28
2,30002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
3,30003,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",28
4,30004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
...,...,...,...
14993,44994,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14994,44995,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14995,44996,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14996,44997,"[1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",28


In [24]:
user_day_df.index = user_day_df['user_id']
user_day_df.drop(columns=['user_id', 'day'], inplace=True)

user_day_df

,period
user_id,
30000,29
30001,28
30002,30
30003,28
30004,30
...,...
44994,30
44995,30
44996,30


In [25]:
# 모든 column 합쳐주기
df1 = df1.groupby('user_id').sum()
dfs = [df1, user_has_multiple_models, user_has_multiple_fwvers, user_day_df]
dfs = dfs[0].join(dfs[1:])
dfs

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,multiple_model,multiple_fwver,period
user_id,,,,,,,,,,,,,,,,,,,,,
30000,2731,0.0,2669.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,0.0,2.0,92.0,0.0,2.0,1,1,29
30001,284,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,28
30002,903,903.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0,1,30
30003,328,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0,1,28
30004,871,0.0,622.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,48.0,90.0,0.0,1,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,1115,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.0,87.0,0.0,0,0,30
44995,515,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,12.0,17.0,0.0,0,1,30
44996,1915,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,329.0,2.0,3.0,0,1,30


In [26]:
test_c = list(dfs.columns)
train_c = list(train.columns)

len(test_c), len(train_c)

(94, 63)

In [27]:
for i in train_c:
    if i not in test_c:
        print(i)

In [28]:
d_columns = []
for i in test_c:
    if i not in train_c:
        d_columns.append(i)
d_columns

['fwver_03.11.1167',
 'fwver_04.16.3553',
 'fwver_04.22.1170',
 'fwver_04.22.1448',
 'fwver_04.22.1478',
 'fwver_04.22.1608',
 'fwver_04.22.1750',
 'fwver_04.22.1772',
 'fwver_04.33.1185',
 'fwver_04.33.1261',
 'fwver_04.73.2237',
 'fwver_04.73.2569',
 'fwver_04.73.2577',
 'fwver_04.82.1684',
 'fwver_04.82.1778',
 'fwver_05.15.2138',
 'fwver_05.66.3237',
 'fwver_10',
 'fwver_10.22.1770',
 'fwver_10.22.1780',
 'errtype_4',
 'errtype_7',
 'errtype_11',
 'errtype_12',
 'errtype_16',
 'errtype_21',
 'errtype_23',
 'errtype_27',
 'errtype_28',
 'errtype_37',
 'multiple_fwver']

In [30]:
dfs.drop(columns=d_columns, inplace=True)

len(dfs.columns), len(train.columns)

(63, 63)

In [31]:
# 스케일링

In [32]:
# multiple_model 만 범주형 데이터

categorical_df = dfs[['multiple_model']]
numerical_df = dfs.drop(columns=['multiple_model'])
numerical_df

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period
user_id,,,,,,,,,,,,,,,,,,,,,
30000,2731,0.0,2669.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,1.0,3.0,0.0,2.0,92.0,0.0,2.0,29
30001,284,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
30002,903,903.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,0.0,14.0,113.0,1.0,2.0,30
30003,328,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,7.0,3.0,1.0,0.0,0.0,40.0,8.0,0.0,28
30004,871,0.0,622.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,5.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,1115,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,30
44995,515,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,1.0,0.0,1.0,0.0,0.0,12.0,17.0,0.0,30
44996,1915,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,802.0,0.0,1.0,1.0,0.0,329.0,2.0,3.0,30


In [33]:
# 수치형 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scalied_df = numerical_df.copy()

scalied_df = scaler.fit_transform(scalied_df)
numerical_df = pd.DataFrame(scalied_df, columns=numerical_df.columns)
numerical_df.head()

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period
0,0.432273,-0.282718,1.465372,-0.138438,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.347687,-0.052885,0.017059,3.792035,-0.133970,0.040193,0.539686,-0.390797,-0.080517,0.269713
1,-0.191518,-0.282718,-0.227667,-0.190118,0.051198,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-1.000390,-0.052885,-0.075341,-1.005075,-0.133970,-0.086729,-0.602594,-0.390797,-0.645212,0.095698
2,-0.033722,0.638161,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,0.347687,-0.052885,-0.075341,0.593961,-0.133970,0.801722,0.800423,-0.334183,-0.080517,0.443729
3,-0.180302,0.051777,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.663371,-0.004461,0.201860,0.593961,-0.133970,-0.086729,-0.105951,0.062114,-0.645212,0.095698
4,-0.041880,-0.282718,0.166889,0.017435,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.073587,-0.018296,0.017059,0.593961,0.288682,-0.086729,-0.006622,4.704445,-0.645212,0.443729


In [34]:
test = numerical_df.join(categorical_df)
test

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,period,multiple_model
0,0.432273,-0.282718,1.465372,-0.138438,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,0.017059,3.792035,-0.133970,0.040193,0.539686,-0.390797,-0.080517,0.269713,NaN
1,-0.191518,-0.282718,-0.227667,-0.190118,0.051198,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,-0.075341,-1.005075,-0.133970,-0.086729,-0.602594,-0.390797,-0.645212,0.095698,NaN
2,-0.033722,0.638161,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,-0.075341,0.593961,-0.133970,0.801722,0.800423,-0.334183,-0.080517,0.443729,NaN
3,-0.180302,0.051777,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.004461,0.201860,0.593961,-0.133970,-0.086729,-0.105951,0.062114,-0.645212,0.095698,NaN
4,-0.041880,-0.282718,0.166889,0.017435,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.018296,0.017059,0.593961,0.288682,-0.086729,-0.006622,4.704445,-0.645212,0.443729,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14993,0.020321,-0.282718,0.479616,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,-0.075341,-1.005075,-0.133970,-0.086729,0.639014,4.534603,-0.645212,0.443729,NaN
14994,-0.132632,0.242479,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.045967,-0.075341,0.593961,-0.133970,-0.086729,-0.453601,0.571638,-0.645212,0.443729,NaN
14995,0.224257,1.670199,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,5.495079,-0.075341,0.593961,0.288682,-0.086729,3.482297,-0.277569,0.201830,0.443729,NaN
14996,6.024724,24.874719,-0.227667,-0.190118,-0.036071,-0.058989,-0.05077,-0.029999,-0.052692,-0.045813,...,-0.052885,-0.075341,0.593961,1.133985,-0.086729,-0.230111,2.383279,-0.645212,0.095698,NaN


In [35]:
test.to_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\test_case1.csv', index=False)